## LSTM Model for parsed at sentence level text (Melisa)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(11)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
"""
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
"""
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.preprocessing.sequence import pad_sequences

import seaborn
seaborn.set()

import tensorflow as tf
#!pip install tensorflow_hub
import tensorflow_hub as hub


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import f1_score
#!pip install gensim
from gensim.models import Word2Vec

In [3]:
#!pip install ipynb
#from ipynb.fs.full.preprocessing.ipynb import process_review_text

In [4]:
train = pd.read_json("../data/train_preprocessed.json")
test = pd.read_json("../data/test.json")

In [5]:
# Adjust based on your vocabulary size
#train_5 = train[:20000]
train_5_1 = train[train["has_spoiler"] == True]
train_5_0 = train[train["has_spoiler"] == False]
train_5 = pd.concat([train_5_1[:5000], train_5_0[:10000]], axis=0)

print(train_5[train_5["has_spoiler"]==True].shape)
print(train_5[train_5["has_spoiler"]==False].shape)



reviews_data = train_5["review_sentences"]
labels = train_5["has_spoiler"]



reviews_data = [' '.join([sentence for _, sentence in review]) for review in reviews_data]


print(labels[labels == True].shape)

#print(reviews_data)
#lengths = [len(i) for i in reviews_data]
#print(min(lengths))
#print(reviews_data[0])

"""
X_np = np.array(reviews_data)

X = tf.convert_to_tensor(X_np)

y = tf.convert_to_tensor(labels)
"""
"""
reviews_np = [np.array(i) for i in reviews_data]
reviews_np = np.array(reviews_np)
labels_np = np.array(labels)
"""
print(reviews_data[1])




(5000, 8)
(10000, 8)
(5000,)
LAST SACRIFICE was amazing. That's all there is to say. **POST-READING EDIT:** It had a bit of the plot disorganization that SPIRIT BOUND did, and I felt like at the end, Lissa seemed a bit unhappy with her future as  she became queen.


In [6]:
max_words = 3000  # Consider only the top 3,000 words in the dataset
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(reviews_data)
sequences = tokenizer.texts_to_sequences(reviews_data)

# Padding
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)






"""
# Train a CBOW model
embedding_dim = 150 # You can adjust this based on your needs
cbow_model = Word2Vec(sentences=sequences, vector_size=embedding_dim, window=5, sg=0, min_count=1, workers=4)

# Save the trained model
cbow_model.save("cbow_model.bin")
"""

'\n# Train a CBOW model\nembedding_dim = 150 # You can adjust this based on your needs\ncbow_model = Word2Vec(sentences=sequences, vector_size=embedding_dim, window=5, sg=0, min_count=1, workers=4)\n\n# Save the trained model\ncbow_model.save("cbow_model.bin")\n'

In [7]:
"""
# Prepare the embedding matrix
word2vec_model = Word2Vec.load("cbow_model.bin")
vocab_size = len(word2vec_model.wv)+1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word2vec_model.wv.key_to_index.items():
    embedding_vector = word2vec_model.wv[word]
    embedding_matrix[i] = embedding_vector
"""

'\n# Prepare the embedding matrix\nword2vec_model = Word2Vec.load("cbow_model.bin")\nvocab_size = len(word2vec_model.wv)+1\nembedding_matrix = np.zeros((vocab_size, embedding_dim))\nfor word, i in word2vec_model.wv.key_to_index.items():\n    embedding_vector = word2vec_model.wv[word]\n    embedding_matrix[i] = embedding_vector\n'

In [8]:
embeddings_path = "../data/glove.6B.50d.txt"
def load_glove_embeddings(embeddings_path):
    embeddings_index = {}
    with open(embeddings_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [9]:
word_index = tokenizer.word_index
vocab_size = min(max_words, len(word_index))
embedding_dim = 50  
embeddings_index = load_glove_embeddings(embeddings_path)

        
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size: 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((padded_sequences, labels))

# Shuffle and batch the dataset
batch_size = 32
buffer_size = 5000  # Set a buffer size for shuffling
dataset = dataset.shuffle(buffer_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [11]:
#embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
#hub_layer = hub.KerasLayer(embedding, dtype=tf.string, trainable=True)

In [12]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(16))

model.add(Dense(1, activation='sigmoid'))

In [13]:
#print(hub_layer(list(reviews_data[0])))

In [14]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

#tf.keras.mixed_precision.set_global_policy('mixed_float16')
#print(labels)

model.fit(padded_sequences, labels, epochs=5, batch_size=64)

Epoch 1/5
235/235 [==============================] - 493s 2s/step - loss: 0.6327 - accuracy: 0.6649 - precision: 0.4420 - recall: 0.0198
Epoch 2/5
235/235 [==============================] - 492s 2s/step - loss: 0.5598 - accuracy: 0.6785 - precision: 0.5276 - recall: 0.3378
Epoch 3/5
235/235 [==============================] - 485s 2s/step - loss: 0.5406 - accuracy: 0.6893 - precision: 0.5444 - recall: 0.4158
Epoch 4/5
235/235 [==============================] - 483s 2s/step - loss: 0.5234 - accuracy: 0.7208 - precision: 0.5971 - recall: 0.4994
Epoch 5/5
235/235 [==============================] - 486s 2s/step - loss: 0.5162 - accuracy: 0.7299 - precision: 0.6086 - recall: 0.5318


In [52]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 emb (InputLayer)            [(None, 1)]               0         
                                                                 
 embedding_4 (Embedding)     (None, 1, 50)             150000    
                                                                 
 lstm_8 (LSTM)               (None, 1, 64)             29440     
                                                                 
 lstm_9 (LSTM)               (None, 16)                5184      
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 184,641
Trainable params: 34,641
Non-trainable params: 150,000
_________________________________________________________________


In [32]:

test_5_1 = test[test["has_spoiler"] == True]
test_5_0 = test[test["has_spoiler"] == False]
test_5 = pd.concat([test_5_1[:5000], test_5_0[:10000]], axis=0)


test_data = test_5["review_sentences"]
test_labels = test_5["has_spoiler"]

In [33]:
 
test_data = [' '.join([sentence for _, sentence in review]) for review in test_data]
""""""

sequences_test = tokenizer.texts_to_sequences(test_data)
# Padding
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

"""
test_reviews_np = [np.array(i) for i in test_data]
test_reviews_np = np.array(test_reviews_np)
test_labels_np = np.array(test_labels)
"""

#predicted_labels = (predictions > 0.5).astype(int)
#print(predicted_labels)




'\ntest_reviews_np = [np.array(i) for i in test_data]\ntest_reviews_np = np.array(test_reviews_np)\ntest_labels_np = np.array(test_labels)\n'

In [34]:

test_loss, test_accuracy, test_precision, test_recall = model.evaluate(padded_sequences_test, test_labels, verbose=0)

''

In [49]:
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1: {2*test_recall*test_precision / (test_recall+test_precision)}")

Test Accuracy: 0.7279999852180481
Test Precision: 0.597046434879303
Test Recall: 0.5659999847412109
Test F1: 0.5811088316522074


In [36]:
predictions = model.predict(padded_sequences_test)

469/469 [==============================] - 122s 261ms/step


In [38]:
model.save_weights("trained_model_lstm", save_format="h5")


In [43]:
from tensorflow.keras import layers, Input, Model
# RANDOM FORESTA FEEDLEMEK İÇİN
def createLSTMModel():
    emb_input = Input(shape=(1,), name="emb")
    embedding = layers.Embedding(input_dim=3000, output_dim=50, input_length=2048,weights=[embedding_matrix], trainable=False)(emb_input)
    lstm1 = layers.LSTM(64, return_sequences=True)(embedding)
    lstm2 = layers.LSTM(16)(lstm1)
    out = layers.Dense(1, activation="sigmoid")(lstm2)
    model = Model(inputs=emb_input, outputs=out)
    return model

model = createLSTMModel()
model.load_weights("trained_model_lstm") # bana attığın train edilmiş model
pretrained_model = tf.keras.Model(
    inputs=model.inputs,
    outputs=model.layers[-2].output
)

X_test = padded_sequences  # train datası
preds = pretrained_model.predict(X_test)

469/469 [==============================] - 126s 268ms/step


In [48]:
preds_np = np.array(preds)
np.save('lstm-preds.npy', preds_np)

In [ ]:
"""
import numpy as np
from tensorflow import keras


# Get class probabilities for the training data
probabilities_model = model.predict(X)
probabilities_model2 = model2.predict(X_train)




#use softmax to convert prediction to probabilities
probabilities_model = keras.activations.softmax(probabilities_model)
probabilities_model2 = keras.activations.softmax(probabilities_model2)


#turn class probabilities to features
combined_features = np.column_stack((probabilities_model1, probabilities_model2))

# Create a random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the random forest classifier
rf_classifier.fit(combined_features, y_train)
"""

In [ ]:
# Assuming 'X_new' is your new data for prediction
"""
#new data
probabilities_new_model1 = model1.predict(X_new)
probabilities_new_model2 = model2.predict(X_new)

probabilities_new_model1 = keras.activations.softmax(probabilities_new_model1)
probabilities_new_model2 = keras.activations.softmax(probabilities_new_model2)

new_data_combined_features = np.column_stack((probabilities_new_model1, probabilities_new_model2))

#predictions using the stacked model
stacked_model_predictions = rf_classifier.predict(new_data_combined_features)
"""